In [8]:
import numpy as np
import scipy.signal

In [3]:
def parse_matrix(s):
    return np.array([[int(n) for n in l] for l in s.splitlines()], dtype=int)

In [70]:
min_filters = [
    np.array([[0, 0, 0], [0, 1, 0], [0, -1, 0]], dtype=int),
    np.array([[0, 0, 0], [0, 1, -1], [0, 0, 0]], dtype=int),
    np.array([[0, 0, 0], [-1, 1, 0], [0, 0, 0]], dtype=int),
    np.array([[0, -1, 0], [0, 1, 0], [0, 0, 0]], dtype=int)
]

def find_minima(height_map):
    minima_1d = [scipy.signal.convolve2d(height_map, f, mode='same', fillvalue=10) < 0 for f in min_filters]
    return np.all(minima_1d, axis=0)

def risk_level(height_map, minima):
    return (height_map[minima] + 1).sum()

In [127]:
def neighbors(x, i, j):
    for di, dj in ((-1, 0), (0, -1), (1, 0), (0, 1)):
        if 0 <= i+di < x.shape[0] and 0 <= j+dj < x.shape[1]:
            yield (i+di, j+dj)

def basin_size(height_map, i, j):
    to_visit = [(i, j)]
    visited = set()
    size = 0
    while to_visit:
        pos = to_visit.pop()
        if pos in visited: continue
        visited.add(pos)
        i, j = pos
        size += 1
        for ni, nj in neighbors(height_map, i, j):
            h = height_map[ni, nj]
            if (ni, nj) in visited or h <= height_map[i, j] or h == 9:
                continue
            to_visit.append((ni, nj))
    return size

def basin_sizes(height_map, minima):
    return [basin_size(height_map, i, j) for i, j in zip(*minima.nonzero())]

In [128]:
height_map = parse_matrix("""2199943210
3987894921
9856789892
8767896789
9899965678""")
height_map
minima = find_minima(height_map)

In [129]:
basin_sizes(height_map, minima)

[3, 9, 14, 9]

In [119]:
with open('../data/day09.txt') as infile:
    height_map = parse_matrix(infile.read())
    minima = find_minima(height_map)
    print('[p1] Risk level:', risk_level(height_map, minima))
    print('[p2] Largest 3 basins:', np.product(sorted(basin_sizes(height_map, minima))[-3:]))

[p1] Risk level: 526
[p2] Largest 3 basins: 1123524
